In [ ]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [ ]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO, DelimitedFiles, Tables, DataFrames
using HypothesisTests

plotly()

### Constantes

In [ ]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Fit global sur l'ensemble des lenslets:

On definit les parametres qui servent a creer le modele  

In [ ]:
#Bbox of the lenslet: box autour d'un set de 3 gaussiennes
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);
a0 = [300.0 , 400.0, 500.0];
fwhm0 = [2.0, 2.0 , 2.0];

#affichage d'un exemple de trois gaussiennes avec la bbox sur les donnees reelles:
f3 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")
data= read(f3[1]);
ldata = data[1012:1018 , 1048:1092]
heatmap(ldata, title = "Lenslet correspondant a la bbox initiale (exemple)")

#il manque les coeff C0 pour pouvoir fiter les gaussiennes, qu'on recupere dans des fichiers txt

### 1 lecture des fichiers txt

On veut recuperer en sortie de fonction la premiere colonne (c0, coef a l'ordre 0, soit le centre de la lenslet).

x(λ) = cx0 + cx1*((λ-λ0)/λ0) + cx2*((λ-λ0)/λ0)^2

y(λ) = cy0 + cy1*((λ-λ0)/λ0) + cy2*((λ-λ0)/λ0)^2

In [ ]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1];
#cx1 = coeffx[:,2];
#cx2 = coeffx[:,3];

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1];
#cy1 = coeffy[:,2];
#cy2 = coeffy[:,3];

#round.(Int, cx0); #conversion de float en int
#round.(Int, cy0);

In [ ]:
typeof(cx0)

### 2 ouvrir et trouver positions dans l'image brute

In [ ]:
#1/ IFS_sim_wave.fits: on ouvre le modele, qui a peu de BP, donc les fit des lenslets seront bons. f5 = FITS("/home/user/Pictures/IFS_sim_wave.fits");
#2/ IFS_calib_wave_corrected: "vrai" fichier de depart (a comparer avec 1/ eventuellement)
f5 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits", "r");

In [ ]:
#Read the data or a subset thereof from disk, and save it in a pre-allocated output array A.
A = Array{Float64}(undef,2048,2048); #pre-allocate matrix A
read!(f5[1], A);
#print(A)

In [ ]:
A

In [ ]:
close(f5);

In [ ]:
#On parcours les lenslets
nb_fit = 5 #test sur quelques lenslets
for i=1:nb_fit #i: numero de la bbox
    #Initialisation
    #xmin = cx0[i,1]-largeur lenslet, ymin = cy0[i,1]-hauteur lenslet
    #xmax = cx0[i,1]+largeur lenslet, ymax = cy0[i,1]+hauteur lenslet
    bbox = BoundingBox(xmin=round.(Int, cx0[i,1])-4,  ymin=round.(Int, cy0[i,1])-22 , xmax=round.(Int, cx0[i,1])+4 , ymax=round.(Int, cy0[i,1])+22 ); #decalage de la bbox avec le fit  
    ainit = [300. , 400. , 500.];
    fwhminit = [2.0, 2.0 , 2.0];
    laser =  LaserModel(λlaser,ainit,fwhminit);
    lenslet = LensletModel(λ0,laser.nλ-1,round(bbox));
    lmod = LensletModel(λ0,laser.nλ-1,bbox);
    Cinit = zeros(2,laser.nλ);
    Cinit[1:2,1:2] = [ [cy0[i,1] 45]; [cx0[i,1] 22] ]; #C0[1:2,1:3] =[ [cy0[i,1] cy1[i,2] cy2[i,3]]; [cx0[i,1] cx1[i,2] cx2[i,3]] ]  
    UpdateDispModel(lmod.dmodel,Cinit); #Update the coefficients of the DispModel.
    UpdateLaserModel(laser,ainit,fwhminit); #Update the amplitude and width of the DispModel.
    
    #Fit du polynome pour chaque lenslet: optimisation
    ldata = view(A, bbox);
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    lkl = LikelihoodIFS(lenslet,laser,ldata); #view(A,lenslet.bbox));
    xopt = vmlmb(likelihood, xinit; verb=50); #penser a rajouter la precision quand on a la carte des BP: view(precision,lenslet.bbox)
    
    #Sauvegarde des valeurs des parametres du modele dans des vecteurs
    #ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
    (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    aopt[i] = aopt;
    fwhm[i] = fwhmopt;
    copt[i] = copt; 
    UpdateDispModel(lmod.dmodel, copt);
    UpdateLaserModel(laser,aopt,fwhmopt);
end


Warning: ajouter un flag de qualité (bon/mauvais ajustements)

Methode 1: tracer des histogrammes de tous les coeff, visualisation des outliers. Ajustement d'une gaussienne, calcul de la moyenne de la variance, ce qui est superieur a 3σ est sorti.

Methode 2: test statistique par rapport au guess si methode 1 marche pas. Histogramme des valeurs de χ2

### 3 affichage

In [ ]:
#loi de dispersion
wxy0 = lmod.dmodel.(wavelengthrange);
(wx0,wy0) = collect.( zip(wxy0...));

In [ ]:
lmod.dmodel.(λlaser) 
ldata = view(A, bbox);
#heatmap(ldata, title = "Fit laser") #1/ pour IFS_sim_wave.fits
heatmap(clamp.(ldata,0,1500), title = "Fit laser")
plot!(wy0,wx0; label="dispersion law")

In [ ]:
@show(ainit,aopt);

@show(fwhminit,fwhmopt);

@show(cinit,copt);

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmod,laser)

#heatmap(limageinit, title = "Initialisation")

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmod,laser)

#heatmap(limageopt, title = "Point d'arrivee de l'optimiseur")


In [ ]:
#Residus
#heatmap( ldata .- limageopt, title = "Residus")